```
This notebook demonstrates a workflow for importing, parsing, and storing email data into a SQL Server database. 

Sample data is from Enron Emails.
```

## Step 1: Import Required Libraries

In [ ]:
import pyodbc
import email.utils
from datetime import datetime
import os
import hashlib

FOLDER = "./test_emails"

## Step 2: Database Connection Configuration

In [ ]:
# Configuration parameters
server = 'Laptop-JLYZH34' 
database = 'Testing_DB02' 
username = 'sa'   
password = 'Welcome@1234'  
timeout = 15               

# Construct connection string
connection_string = f'''
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
Timeout={timeout};
'''

# Test database connection
def test_connection():
    try:
        with pyodbc.connect(connection_string) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT @@VERSION")
            version = cursor.fetchone()[0]
            print(f"SQL Server Version: {version[:50]}...")
            return True
    except Exception as e:
        return e

test_connection()

SQL Server Version: Microsoft SQL Server 2022 (RTM-GDR) (KB5065221) - ...


True

## Step 3: Create Database Tables

In [25]:
# Create database tables
def create_email_tables():
    
    with pyodbc.connect(connection_string) as conn:
        cursor = conn.cursor()
        
        try:
            # Create schema
            cursor.execute("""
                IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'email_data')
                BEGIN
                    EXEC('CREATE SCHEMA email_data')
                END
            """)
            
            # Create main email table
            cursor.execute("""
                IF OBJECT_ID('email_data.emails') IS NULL
                BEGIN
                    CREATE TABLE email_data.emails (
                        id INT IDENTITY(1,1) PRIMARY KEY,
                        message_id NVARCHAR(500),
                        date_sent DATETIME,
                        from_address NVARCHAR(255),
                        to_addresses NVARCHAR(MAX),
                        cc_addresses NVARCHAR(MAX),
                        subject NVARCHAR(1000),
                        body_text NVARCHAR(MAX),
                        x_folder NVARCHAR(500),
                        x_origin NVARCHAR(255),
                        x_filename NVARCHAR(255),
                        email_hash NVARCHAR(64),
                        import_date DATETIME DEFAULT GETDATE()
                    )
                    
                    -- Create indexes
                    CREATE INDEX IX_emails_message_id ON email_data.emails (message_id)
                    CREATE INDEX IX_emails_from_address ON email_data.emails (from_address)
                    CREATE INDEX IX_emails_date_sent ON email_data.emails (date_sent)
                    CREATE INDEX IX_emails_hash ON email_data.emails (email_hash)
                END
            """)
            
            conn.commit()
            print("Database tables created successfully!")

        except Exception as e:
            print(f"Failed to create tables: {e}")
            conn.rollback()

# Execute table creation
create_email_tables()

Database tables created successfully!


## Step 4: Email Parser Class

In [26]:
# Email parser class
class EmailParser:
    def __init__(self, email_content):
        self.raw_content = email_content
        self.message_id = None
        self.date_sent = None
        self.from_address = None
        self.to_addresses = None
        self.cc_addresses = None
        self.subject = None
        self.body_text = ""
        self.x_folder = None
        self.x_origin = None
        self.x_filename = None
        self.email_hash = None
        
        # Parse email content
        self.parse_email()
        # Generate hash
        self.generate_hash()
    
    def parse_email(self):
        lines = self.raw_content.split('\n')
        in_body = False
        current_header = None
        current_value = ""
        
        for line in lines:
            line = line.rstrip()
            
            # Empty line indicates end of headers, start of body
            if line == "" and not in_body:
                if current_header:
                    self.set_header_value(current_header, current_value.strip())
                    current_header = None
                    current_value = ""
                in_body = True
                continue

            # Body content
            if in_body:
                self.body_text += line + "\n"
            
            # Header content
            else:
                # Multi-line header continuation
                if line.startswith((' ', '\t')) and current_header:
                    current_value += " " + line.strip()
                
                # New header line
                else:
                    # Save previous header
                    if current_header:
                        self.set_header_value(current_header, current_value.strip())
                    
                    # Parse new header
                    if ':' in line:
                        header_name, header_value = line.split(':', 1)
                        current_header = header_name.strip()
                        current_value = header_value.strip()
                    else:
                        current_header = None
                        current_value = ""
        
        # Process last header (if email has no body)
        if current_header and not in_body:
            self.set_header_value(current_header, current_value.strip())
        
        # Clean body text
        self.body_text = self.body_text.strip()
        
        # Parse date
        self.parse_date()
    
    def set_header_value(self, header_name, header_value):
        header_name = header_name.lower()
        
        if header_name == 'message-id':
            # Keep only actual message-id, remove angle brackets
            self.message_id = header_value.strip('<>')
        elif header_name == 'date':
            self.date_raw = header_value
        elif header_name == 'from':
            self.from_address = header_value
        elif header_name == 'to':
            self.to_addresses = header_value
        elif header_name == 'cc':
            self.cc_addresses = header_value
        elif header_name == 'subject':
            self.subject = header_value
        elif header_name == 'x-folder':
            self.x_folder = header_value
        elif header_name == 'x-origin':
            self.x_origin = header_value
        elif header_name == 'x-filename':
            self.x_filename = header_value
    
    def parse_date(self):
        if hasattr(self, 'date_raw') and self.date_raw:
            try:
                date_tuple = email.utils.parsedate_tz(self.date_raw)
                if date_tuple:
                    timestamp = email.utils.mktime_tz(date_tuple)
                    self.date_sent = datetime.fromtimestamp(timestamp)
            except Exception as e:
                print(f"Date parsing failed: {e}")
                self.date_sent = None
    
    # pending on discussion
    def generate_hash(self):
        content_parts = [
            str(self.message_id or ""),
            str(self.from_address or ""),
            str(self.subject or ""),
            str(self.body_text or "")
        ]
        content = "".join(content_parts)
        self.email_hash = hashlib.md5(content.encode('utf-8')).hexdigest()

print("Email parser class defined successfully")

Email parser class defined successfully


## Step 5: Database Save Functions

In [27]:
# Database save functions
def save_email_to_database(email_parser):
    
    with pyodbc.connect(connection_string) as conn:
        cursor = conn.cursor()
        
        try:
            # Check if email with same hash already exists (deduplication)
            cursor.execute("""
                SELECT COUNT(*) FROM email_data.emails 
                WHERE email_hash = ?
            """, email_parser.email_hash)
            
            if cursor.fetchone()[0] > 0:
                print(f"Email already exists, skipping: {email_parser.subject}")
                return False
            
            # Insert main email record
            cursor.execute("""
                INSERT INTO email_data.emails (
                    message_id, date_sent, from_address, to_addresses, 
                    cc_addresses, subject, body_text, x_folder, 
                    x_origin, x_filename, email_hash
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                email_parser.message_id,
                email_parser.date_sent,
                email_parser.from_address,
                email_parser.to_addresses,
                email_parser.cc_addresses,
                email_parser.subject,
                email_parser.body_text,
                email_parser.x_folder,
                email_parser.x_origin,
                email_parser.x_filename,
                email_parser.email_hash
            ))
            
            # Get inserted email ID
            cursor.execute("SELECT @@IDENTITY")
            
            conn.commit()
            print(f"Email saved successfully: {email_parser.subject}")
            return True
            
        except Exception as e:
            print(f"Failed to save email: {e}")
            conn.rollback()
            return False


print("Database save functions defined successfully")

Database save functions defined successfully


## Step 6: Import Email Data

In [28]:
# Single email file
def import_email_file(file_path):
    
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return False
    
    try:
        # Try multiple encodings to read file
        encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1252', 'iso-8859-1']
        file_content = None
        used_encoding = None
        
        for encoding in encodings:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    file_content = f.read()
                used_encoding = encoding
                break
            except UnicodeDecodeError:
                continue
        
        if file_content is None:
            print("Failed to read file with all encoding attempts")
            return False
        
        print(f"File read successfully, encoding: {used_encoding}")
        print(f"File size: {len(file_content)} characters")
        
        # Parse email
        parser = EmailParser(file_content)
        
        # Save to database
        if save_email_to_database(parser):
            print("\nEmail import successful!")
            return True
        else:
            print("\nEmail save failed!")
            return False
        
    except Exception as e:
        return e

# Batch import from folder
def import_email_folder(folder_path):
    
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return
    
    # Find all email files
    email_files = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.txt', '.eml', '.msg')):
            email_files.append(os.path.join(folder_path, file))
    
    if not email_files:
        print(f"No email files found in folder: {folder_path}")
        return
    
    print(f"Found {len(email_files)} email files")
    print("=" * 50)
    
    success_count = 0
    for i, file_path in enumerate(email_files, 1):
        print(f"\nProcessing file {i}/{len(email_files)}: {os.path.basename(file_path)}")
        if import_email_file(file_path):
            success_count += 1
    
    print("\n" + "=" * 50)
    print(f"Batch import complete! Success: {success_count}/{len(email_files)}")


# Test folder path and execute import
print(f"Folder exists: {os.path.exists(FOLDER)}")
import_email_folder(FOLDER)

Folder exists: True
Found 2 email files

Processing file 1/2: test_1.txt
File read successfully, encoding: utf-8
File size: 1339 characters
Email already exists, skipping: NGI Publications - Thursday, 14 December 2000

Email save failed!

Processing file 2/2: test_2.txt
File read successfully, encoding: utf-8
File size: 1730 characters
Email already exists, skipping: Re:

Email save failed!

Batch import complete! Success: 0/2
